In [94]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import seaborn as sns

plt.rcParams['figure.figsize'] = 8, 7
plt.rcParams["font.weight"] = "bold"
plt.rcParams["font.family"] = "normal"
plt.rcParams["font.size"] = 25
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams['xtick.labelsize']=18
plt.rcParams['ytick.labelsize']=18
plt.rcParams['axes.labelsize']=18
plt.rcParams['legend.fontsize']: 16
    
sns.set()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [95]:
### Setting Up DB Connection ###
db = 'database.sqlite'
conn = sqlite3.connect(db)

### Extracting All Available Tables From DB ###
db_tables = pd.read_sql("""SELECT * 
                            FROM sqlite_master
                            WHERE type='table';""", conn)
print(db_tables)

    type               name           tbl_name  rootpage                                                sql
0  table    sqlite_sequence    sqlite_sequence         4             CREATE TABLE sqlite_sequence(name,seq)
1  table  Player_Attributes  Player_Attributes        11  CREATE TABLE "Player_Attributes" (\n\t`id`\tIN...
2  table             Player             Player        14  CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3  table              Match              Match        18  CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4  table             League             League        24  CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5  table            Country            Country        26  CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6  table               Team               Team        29  CREATE TABLE "Team" (\n\t`id`\tINTEGER PRIMARY...
7  table    Team_Attributes    Team_Attributes         2  CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


 **^^ Here we see we have 7 tables in total we'll be interested in exploring further (Player_Attributes, Player, Match, League, Country, Team, Team_Attributes)**

In [96]:
### Exploring Features in Each Table ###
for table in db_tables.tbl_name:
    if table != 'sqlite_sequence':
        print('\n', table + ': ')
        new = pd.read_sql("""SELECT * 
                            FROM """ + table + """ LIMIT 2""", conn)
        print(new.columns)
                        


 Player_Attributes: 
Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes'], dtype='object')

 Player: 
Index(['id', 'player_api_id', 'player_name', 'player_fifa_api_id', 'birthday', 'height', 'weight'], dtype='object')

 Match: 
Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD'

# Countries/Leagues/Teams

In [97]:
# Countries in DB #
countries = pd.read_sql("""SELECT DISTINCT name AS Country
                            FROM Country""", conn)
print(countries)

        Country
0       Belgium
1       England
2        France
3       Germany
4         Italy
5   Netherlands
6        Poland
7      Portugal
8      Scotland
9         Spain
10  Switzerland


In [98]:
# Leagues in each country # 
league_country = pd.read_sql("""SELECT c.name AS Country, l.name AS League
                            FROM League AS l
                            INNER JOIN Country AS c
                            ON l.country_id = c.id""", conn)
print(league_country)

        Country                    League
0       Belgium    Belgium Jupiler League
1       England    England Premier League
2        France            France Ligue 1
3       Germany     Germany 1. Bundesliga
4         Italy             Italy Serie A
5   Netherlands    Netherlands Eredivisie
6        Poland        Poland Ekstraklasa
7      Portugal  Portugal Liga ZON Sagres
8      Scotland   Scotland Premier League
9         Spain           Spain LIGA BBVA
10  Switzerland  Switzerland Super League


In [99]:
# Average Number of Matches Played in Each League per Season (08/09 - 15/16)# 
avg_games = pd.read_sql("""SELECT c.name AS Country, l.name AS League, (COUNT(c.name) / COUNT(DISTINCT m.season)) AS Avg_Games_Season
                                FROM Match as m
                                INNER JOIN Country as c
                                ON m.country_id = c.id
                                INNER JOIN League as l
                                ON m.league_id = l.id
                                GROUP BY c.name
                                ORDER BY (COUNT(c.name) / COUNT(DISTINCT m.season)) DESC""", conn)
print(avg_games)

        Country                    League  Avg_Games_Season
0         Spain           Spain LIGA BBVA               380
1        France            France Ligue 1               380
2       England    England Premier League               380
3         Italy             Italy Serie A               377
4   Netherlands    Netherlands Eredivisie               306
5       Germany     Germany 1. Bundesliga               306
6      Portugal  Portugal Liga ZON Sagres               256
7        Poland        Poland Ekstraklasa               240
8      Scotland   Scotland Premier League               228
9       Belgium    Belgium Jupiler League               216
10  Switzerland  Switzerland Super League               177


In [100]:
### Number of Teams Per League ###
team_count = pd.read_sql("""SELECT c.name AS Country, l.name AS League, COUNT(DISTINCT m.home_team_api_id) AS Num_Teams
                                FROM Match as m
                                INNER JOIN Country as c
                                ON m.country_id = c.id
                                INNER JOIN League as l
                                ON m.league_id = l.id
                                GROUP BY c.name 
                                ORDER BY COUNT(DISTINCT m.home_team_api_id) DESC""", conn)
print(team_count)

        Country                    League  Num_Teams
0        France            France Ligue 1         35
1       England    England Premier League         34
2         Spain           Spain LIGA BBVA         33
3         Italy             Italy Serie A         32
4       Germany     Germany 1. Bundesliga         30
5      Portugal  Portugal Liga ZON Sagres         29
6   Netherlands    Netherlands Eredivisie         25
7       Belgium    Belgium Jupiler League         25
8        Poland        Poland Ekstraklasa         24
9      Scotland   Scotland Premier League         17
10  Switzerland  Switzerland Super League         15


In [101]:
### Champions of Each Season for Each League ###
matches = pd.read_sql("""SELECT m.season, l.name, m.home_team_api_id, m.away_team_api_id, m.home_team_goal, m.away_team_goal
                            FROM Match as m
                            INNER JOIN League as l
                            ON m.league_id = l.id""", conn)
print(matches.head(5))

# add win/loss/tie points for both home and away #
matches['home_points'] = ''
matches['away_points'] = ''
for idx, row in matches.iterrows():
    if row['home_team_goal'] > row['away_team_goal']:
        matches.loc[idx, 'home_points'] = 3
        matches.loc[idx, 'away_points'] = 0
    elif row['home_team_goal'] < row['away_team_goal']:
        matches.loc[idx, 'home_points'] = 0
        matches.loc[idx, 'away_points'] = 3
    else:
        matches.loc[idx, 'home_points'] = 1
        matches.loc[idx, 'away_points'] = 1

print('\nAdding points awarded to each team after game', '\n')
print(matches[['home_team_goal', 'away_team_goal', 'home_points', 'away_points']].head(5))

      season                    name  home_team_api_id  away_team_api_id  home_team_goal  away_team_goal
0  2008/2009  Belgium Jupiler League              9987              9993               1               1
1  2008/2009  Belgium Jupiler League             10000              9994               0               0
2  2008/2009  Belgium Jupiler League              9984              8635               0               3
3  2008/2009  Belgium Jupiler League              9991              9998               5               0
4  2008/2009  Belgium Jupiler League              7947              9985               1               3

Adding points awarded to each team after game 

   home_team_goal  away_team_goal  home_points  away_points
0               1               1            1            1
1               0               0            1            1
2               0               3            0            3
3               5               0            3            0
4               1    

In [102]:
### Assigning Team Names based on api_id ###

teams = pd.read_sql("""SELECT team_api_id, team_long_name
                        FROM Team""", conn)

for idx, row in matches.iterrows():
    for idx2, row2 in teams.iterrows():
        if row['home_team_api_id'] == row2['team_api_id']:
            matches.loc[idx, 'home_team_api_id'] = row2['team_long_name']
        elif row['away_team_api_id'] == row2['team_api_id']:
            matches.loc[idx, 'away_team_api_id'] = row2['team_long_name']

matches.columns = ['season', 'league', 'home_team', 'away_team', 'home_team_goal', 'away_team_goal', 'home_points', 'away_points']
print(matches.head(3))

      season                  league          home_team         away_team  home_team_goal  away_team_goal  home_points  away_points
0  2008/2009  Belgium Jupiler League           KRC Genk      Beerschot AC               1               1            1            1
1  2008/2009  Belgium Jupiler League   SV Zulte-Waregem  Sporting Lokeren               0               0            1            1
2  2008/2009  Belgium Jupiler League  KSV Cercle Brugge    RSC Anderlecht               0               3            0            3


In [103]:
home = matches.groupby(['season', 'league', 'home_team'])['home_points'].sum()

print('Home points for each team')
print(home)

Home points for each team
season     league                    home_team              
2008/2009  Belgium Jupiler League    Beerschot AC               30
                                     Club Brugge KV             35
                                     FCV Dender EH              19
                                     KAA Gent                   30
                                     KRC Genk                   26
                                                                ..
2015/2016  Switzerland Super League  FC Thun                    23
                                     FC Vaduz                   20
                                     FC Zürich                  19
                                     Grasshopper Club Zürich    30
                                     Lugano                     22
Name: home_points, Length: 1478, dtype: int64


In [104]:
away = matches.groupby(['season', 'league', 'away_team'])['away_points'].sum()

print('Away points for each team')
print(away)

Away points for each team
season     league                    away_team              
2008/2009  Belgium Jupiler League    Beerschot AC               12
                                     Club Brugge KV             24
                                     FCV Dender EH              16
                                     KAA Gent                   29
                                     KRC Genk                   24
                                                                ..
2015/2016  Switzerland Super League  FC Thun                    18
                                     FC Vaduz                   16
                                     FC Zürich                  15
                                     Grasshopper Club Zürich    23
                                     Lugano                     13
Name: away_points, Length: 1478, dtype: int64
